In [24]:
import torch
import torch.nn as nn

In [2]:
bsz = 3
seqlen= 4
num_experts = 3
dim = 1024
rank=16


In [3]:
x = torch.randn(bsz,seqlen,dim)

In [4]:
linearA = nn.Linear(dim,rank,bias=False)
linearB = nn.Linear(rank,dim,bias=False)
core1 = nn.Parameter(torch.randn(rank,rank))
core2 = nn.Parameter(torch.randn(rank,rank))
core3 = nn.Parameter(torch.randn(rank,rank))
router = nn.Linear(dim,num_experts)

In [5]:
core1

Parameter containing:
tensor([[-3.0672e-01,  9.6503e-01,  8.7366e-01,  1.6000e+00,  1.3749e+00,
          6.1006e-01,  1.0057e+00,  2.1399e-01,  3.2267e-01,  2.6187e-01,
         -1.4546e+00,  6.9437e-01,  4.0496e-01, -1.3593e+00, -4.3120e-01,
         -6.0866e-01],
        [-1.0716e-01,  8.8604e-01, -2.5447e-01, -1.7907e-01,  3.6150e-01,
          7.3252e-01,  1.3947e+00, -7.2029e-01, -2.8575e-01,  3.4056e-01,
         -9.1186e-01, -7.9203e-01,  9.9869e-01, -1.3945e+00, -2.5895e-02,
         -9.1085e-02],
        [ 6.3443e-01, -1.0613e+00,  1.3626e+00, -9.7046e-01,  8.8673e-02,
         -1.4458e+00, -9.4218e-01, -2.0670e+00,  4.7041e-01,  1.5380e-01,
          3.2222e+00,  1.5429e+00, -6.8885e-01, -2.7969e-01, -9.7573e-01,
         -6.8406e-01],
        [ 1.8071e+00,  8.5823e-01, -1.0239e+00, -1.4315e-01,  4.0643e-01,
          6.5112e-01,  1.7962e+00,  1.3488e+00, -8.6346e-01, -1.1777e+00,
          3.5909e-01, -3.1651e-01,  1.0416e+00,  2.9458e-01,  5.1505e-01,
          1.0371e+00]

In [6]:
router_weight = router(x)

In [7]:
router_weight.shape

torch.Size([3, 4, 3])

In [8]:
t = linearA(x)
h = linearB(t @ core1) * router_weight[:,:,0].unsqueeze(-1) + linearB(t @ core2) * router_weight[:,:,1].unsqueeze(-1) + linearB(t @ core3) * router_weight[:,:,2].unsqueeze(-1)

In [9]:
h

tensor([[[ 0.2102, -0.1169, -0.0297,  ..., -0.4330,  0.2857, -1.0602],
         [-0.3059, -0.1817,  1.3140,  ...,  1.0026,  0.1188, -1.5246],
         [ 1.3982, -0.0305, -0.2875,  ...,  0.2336, -1.2575,  0.2883],
         [ 0.8093, -0.3925, -0.0798,  ..., -0.3715,  0.3894, -2.2035]],

        [[ 2.1704, -0.7111,  0.8006,  ..., -0.6447,  0.8944, -2.1577],
         [-0.2223,  0.3207, -0.0949,  ...,  0.0709,  0.3278,  0.3170],
         [-0.8162,  0.5940, -2.5062,  ...,  1.0938, -1.5720,  2.1152],
         [-1.4782,  0.1252, -0.4902,  ...,  0.3066,  0.2931,  1.9902]],

        [[ 0.6346, -0.1035,  0.2022,  ...,  0.2260, -0.0588,  0.8885],
         [ 0.6743, -1.1728,  0.3213,  ..., -3.0695,  1.4022, -1.0476],
         [-0.4383,  0.8446,  0.2500,  ...,  1.3112,  0.7454, -1.1052],
         [-0.2573,  0.5427,  2.6926,  ..., -3.1439,  1.5082, -3.9849]]],
       grad_fn=<AddBackward0>)

In [10]:
test = core1 * router_weight[:,:,0].unsqueeze(-1).unsqueeze(-1)

In [11]:
test.shape

torch.Size([3, 4, 16, 16])

In [12]:
core = (core1 * router_weight[:,:,0].unsqueeze(-1).unsqueeze(-1) + core2 * router_weight[:,:,1].unsqueeze(-1).unsqueeze(-1) + core3 * router_weight[:,:,2].unsqueeze(-1).unsqueeze(-1))

In [21]:
h2_=torch.einsum('bs...r,bsrr->bsr',linearA(x), core)

In [22]:
h3_ = linearB(h2_)

In [23]:
h3_

tensor([[[ 0.0899, -0.1224,  0.1761,  ..., -0.0160, -0.0660,  0.0016],
         [-0.2245,  0.0893,  0.3942,  ...,  0.2365,  0.1889, -0.4225],
         [ 0.2494,  0.0432, -0.0520,  ...,  0.4073, -0.5017,  0.1072],
         [ 0.0358,  0.0665, -0.0907,  ...,  0.0876,  0.1728, -0.3545]],

        [[ 0.0493, -0.2844,  0.3351,  ..., -0.3415, -0.0746,  0.1203],
         [ 0.0375, -0.0381,  0.1448,  ..., -0.1325, -0.0077, -0.1185],
         [ 0.5074, -0.0093, -0.2681,  ..., -0.1647,  0.2090, -0.3201],
         [-0.0860,  0.0254, -0.5743,  ...,  0.2762, -0.0057, -0.2561]],

        [[-0.0886, -0.0149, -0.0914,  ..., -0.0422,  0.0252,  0.2563],
         [ 0.6582,  0.1026,  0.1243,  ..., -0.0930,  0.3214, -0.6203],
         [ 0.0479, -0.1637, -0.2453,  ..., -0.0193,  0.0276, -0.5304],
         [ 0.6821, -0.6126,  0.2182,  ..., -1.2273, -0.2471,  0.2422]]],
       grad_fn=<UnsafeViewBackward0>)

In [13]:
core.shape

torch.Size([3, 4, 16, 16])

In [17]:
h2 = linearA(x).unsqueeze(-2) @ core

In [18]:
h2.shape

torch.Size([3, 4, 1, 16])

In [19]:
h3 = linearB(h2.squeeze(-2))

In [20]:
h3

tensor([[[ 0.2102, -0.1169, -0.0297,  ..., -0.4330,  0.2857, -1.0602],
         [-0.3059, -0.1817,  1.3140,  ...,  1.0026,  0.1188, -1.5246],
         [ 1.3982, -0.0305, -0.2875,  ...,  0.2336, -1.2575,  0.2883],
         [ 0.8093, -0.3925, -0.0798,  ..., -0.3715,  0.3894, -2.2035]],

        [[ 2.1704, -0.7111,  0.8006,  ..., -0.6447,  0.8944, -2.1577],
         [-0.2223,  0.3207, -0.0949,  ...,  0.0709,  0.3278,  0.3170],
         [-0.8162,  0.5940, -2.5062,  ...,  1.0938, -1.5720,  2.1152],
         [-1.4782,  0.1252, -0.4902,  ...,  0.3066,  0.2931,  1.9902]],

        [[ 0.6346, -0.1035,  0.2022,  ...,  0.2260, -0.0588,  0.8885],
         [ 0.6743, -1.1728,  0.3213,  ..., -3.0695,  1.4022, -1.0476],
         [-0.4383,  0.8446,  0.2500,  ...,  1.3112,  0.7454, -1.1052],
         [-0.2573,  0.5427,  2.6926,  ..., -3.1439,  1.5082, -3.9849]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
linearA(x).shape

torch.Size([3, 4, 16])

: 

In [ ]:
h2.shape

torch.Size([3, 4, 1, 16])

: 

In [ ]:
linearA(x).unsqueeze(-2).shape

torch.Size([3, 4, 1, 16])

: 

In [31]:
a = torch.load('/data/workspace/projects/moe/AdaMoLE/outputs/qwen3-8b-adamole-the8-math-14k/adapter_model.bin')

In [33]:
a.keys()

dict_keys(['base_model.model.model.layers.0.self_attn.q_proj.lora_A.0.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.1.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.2.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.3.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.4.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.5.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.6.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.7.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.0.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.1.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.2.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.3.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.4.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.5.weight', 'base_model.model.model.layers.0.self

In [35]:
a['base_model.model.model.layers.0.self_attn.q_proj.moe_layer.experts.0.lora_A.weight']

tensor([[-1.2396e-02, -7.1400e-03,  2.5368e-03,  ...,  9.8336e-03,
         -1.1732e-02,  1.2827e-02],
        [-1.4368e-02, -1.8441e-02, -1.4508e-02,  ..., -4.0336e-04,
         -9.8678e-03, -3.7088e-03],
        [-1.3828e-02, -2.2106e-03, -9.0081e-03,  ..., -8.4081e-03,
          9.5567e-03, -1.4297e-02],
        ...,
        [ 6.6728e-04,  1.6089e-02,  4.6001e-03,  ..., -5.6628e-03,
          9.2418e-03, -6.6142e-04],
        [ 5.5787e-03,  3.4591e-03, -1.3449e-04,  ...,  8.4715e-05,
          3.5870e-03,  1.5209e-02],
        [-7.9833e-03,  7.4099e-03,  9.2322e-03,  ..., -4.9362e-03,
          9.3144e-03, -1.4914e-02]], device='cuda:0')

In [36]:
a['base_model.model.model.layers.0.self_attn.q_proj.lora_A.0.weight']

tensor([[-1.2396e-02, -7.1400e-03,  2.5368e-03,  ...,  9.8336e-03,
         -1.1732e-02,  1.2827e-02],
        [-1.4368e-02, -1.8441e-02, -1.4508e-02,  ..., -4.0336e-04,
         -9.8678e-03, -3.7088e-03],
        [-1.3828e-02, -2.2106e-03, -9.0081e-03,  ..., -8.4081e-03,
          9.5567e-03, -1.4297e-02],
        ...,
        [ 6.6728e-04,  1.6089e-02,  4.6001e-03,  ..., -5.6628e-03,
          9.2418e-03, -6.6142e-04],
        [ 5.5787e-03,  3.4591e-03, -1.3449e-04,  ...,  8.4715e-05,
          3.5870e-03,  1.5209e-02],
        [-7.9833e-03,  7.4099e-03,  9.2322e-03,  ..., -4.9362e-03,
          9.3144e-03, -1.4914e-02]], device='cuda:0')

In [38]:
a.keys().__len__()

6300